# <span style='color:SteelBlue'>P5 - Segmentez des clients d'un site e-commerce (part 3)</span>

<img src="https://asset.brandfetch.io/idvSn4Org5/idCJV4h5Ot.png" width="400" height="400">

# <span style='background:white; color:black'>Sommaire</span>

Importations des librairies utilisées dans ce notebook <br>

Paramètres d'affichage du notebook <br>

Fonctions utilisées dans ce notebook <br>

# <span style='background:red; color:white'>A METTRE A JOUR</span>

**Etape 1 : Description des données**


Conclusion

# <span style='background:blue'>Introduction</span>

Olist souhaite que nous fournissons à ses équipes d'e-commerce une segmentation des clients qu’elles pourront utiliser au quotidien pour leurs campagnes de communication.

Nous devons fournir à l’équipe Marketing une description actionable de votre segmentation et de sa logique sous-jacente pour une utilisation optimale, ainsi qu’une proposition de contrat de maintenance basée sur une analyse de la stabilité des segments au cours du temps.
___

**Mission**

**Réaliser la segmentation des clients en utilisant des méthodes non-superviséees**. Notre objectif est de comprendre les différents types d’utilisateurs grâce à leur comportement et à leurs données personnelles. Pour cela nous devons :
1. Réaliser une courte analyse exploratoire.
2. Tester différents modèles de segmentation afin de répondre au mieux à la problématique.
3. Créer une proposition de contrat de maintenance basée sur une analyse de la stabilité des segments au cours du temps.

Ce notebook présente la 3ème partie. Nous réalisons une simulation pour déterminer la stabilité des segments au cours du temps pour émettre une proposition de fréquence de maintenance au cours du temps.
___

**Source des datasets**<br>
### <span style='background:red; color:white'>A METTRE A JOUR</span>

Olist nous fournit une base de données anonymisée comportant des informations sur l’historique de commandes, les produits achetés, les commentaires de satisfaction, et la localisation des clients depuis janvier 2017.

Ce dataset a été nettoyé et enrichi dans le notebook 1. Nous allons donc utiliser le dataset issu du notebook 1 pour réaliser la segmentation des clients.

nom du fichier : **olist_clients_segmentation.csv**

# <span style='background:grey'>Importations des librairies utilisées dans ce notebook</span>

In [2]:
# Librairies
import os.path
import sys
import matplotlib as mpl
import matplotlib.pyplot as plt
from matplotlib.ticker import FuncFormatter
import pandas as pd
import seaborn as sns
import numpy as np
import time
from IPython.display import display
import scipy
from scipy.stats import shapiro
from datetime import datetime

# Affichage plot dans le notebook
%matplotlib inline

# Versions
print('Version des librairies utilisées :')
print('Python        : ' + sys.version)
print('Matplotlib    : ' + mpl.__version__)
print('NumPy         : ' + np.version.full_version)
print('Pandas        : ' + pd.__version__)
print('Scipy         : ' + scipy.__version__)
print('Seaborn       : ' + sns.__version__)

# Afficher heure lancement
maintenant = datetime.now().isoformat()
print('\nCode lancé le : ' + maintenant)

# Enregistrer l'heure de début
start_time = time.time()

Version des librairies utilisées :
Python        : 3.11.6 (tags/v3.11.6:8b6ee5b, Oct  2 2023, 14:57:12) [MSC v.1935 64 bit (AMD64)]
Matplotlib    : 3.8.2
NumPy         : 1.26.2
Pandas        : 2.1.3
Scipy         : 1.11.4
Seaborn       : 0.13.0

Code lancé le : 2023-12-05T12:32:32.061249


# <span style='background:grey'>Paramètres d'affichage du notebook</span>

In [3]:
# paramétrage des paramètres d'affichage dans le notebook (pour afficher les définitions des indicateurs)
pd.set_option('display.max_row', 100)
pd.set_option('display.max_column', 100)
pd.set_option('display.precision', 5)

# Création d'1 palette de couleurs
colors_list = ['Gold', 'MediumPurple', 'DarkOrange', 'LimeGreen', 'Tomato', 'SteelBlue', 'SaddleBrown', 'DeepPink']

# Définit le style de la police pour le titre des graphiques
font_title = {'family': 'Times New Roman',
                'color': '000000',  # Noir en notation hexadécimale
                'size': 22}

# Définit le style de la police pour les axes du graphique
font_axe = {'family': 'Times New Roman',
                'color': '000000',  # Noir en notation hexadécimale
                'size': 16}

# Seuil de significativité des tests statistiques
seuil_significativite = 0.05

# <span style='background:grey'>Fonctions utilisées dans ce notebook</span>

# <span style='background:grey'>Importation des données</span>

Importation du fichiers CSV avec Pandas

In [ ]:
# Définir le chemin
chemin = "./data/cleaned/"

# Obtenir une liste de tous les fichiers dans le répertoire
fichiers = os.listdir(chemin)

# Initialiser un dictionnaire vide et le print de la liste des fichiers importés
data_brut = {}
print('===> Liste des fichiers importés: <====\n')

# Boucle sur chaque fichier
for fichier in fichiers:
    
    # Traiter uniquement les fichiers .csv
    if fichier.endswith('.csv'):
        print(fichier)
        
        # Lire le fichier csv et le stocker dans le dictionnaire
        data_brut[fichier] = pd.read_csv(chemin + fichier, sep=",", encoding='utf-8')

# <span style='background:blue'>Etape 1 : ...</span>